In [ ]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing

# 统计模型与统计要素

## 时间序列分析常用统计模型

单变量时间序列统计学模型，如：平均方法、平滑方法、有/无季节性条件的 ARIMA 模型。

多变量时间序列统计学模型，如：外生回归变量、VAR。

附加或组件模型，如：Facebook Prophet、ETS。

结构化时间序列模型，如：贝叶斯结构化时间序列模型、分层时间序列模型。

在本篇文章中，我们主要关注 SARIMA 和 Holt-winters 方法。
## 单变量时间序列统计学模型的关键要素

如果我们想要对时间序列数据进行上述统计学模型分析，需要进行一系列处理使得：
(1)数据均值 
(2)数据方差 
(3)数据自协方差 
这三个指标不依赖于时间项。即时间序列数据具有平稳性。
如何明确时间序列数据是否具有平稳性？
可以从两个特征进行判断。
(1) 趋势，即均值随时间变化；
(2) 季节性，即方差随时间变化、自协方差随时间变化。

## 若满足以上两个条件，则时间序列数据不符合平稳性要求。

可以通过以下方法消除上述问题：
1. 变换，如：取对数、取平方等。
2. 平滑处理，如：移动平均等。
3. 差分。
4. 分解。
5. 多项式拟合，如：拟合回归。

## ARIMA：差分整合移动平均自回归模型


Autoregressive Integrated Moving Average model (ARIMA)，差分整合移动平均自回归模型。ARIMA(p,d,q)主要包含三项：
p:AR项，即自回归项(autoregression)，将时间序列下一阶段描述为前一阶段数据的线性映射。 
d项，即积分项(integration)，时间序列的差分预处理步骤，使其满足平稳性要求  
q:MA项，即移动平均项(moving average)，将时间序列下一阶段描述为前一阶段数据平均过程中的残留误差的线性映射。

该模型需要指定 p d q 三项参数，并按照顺序执行。ARIMA 模型也可以用于开发 AR, MA 和 ARMA 模型。



## ACF 和 PACF 图

自相关函数，autocorrelation function(ACF)，描述了时间序列数据与其之后版本的相关性（如：Y(t) 与 Y(t-1) 之间的相关性）。
偏自相关函数，partial autocorrelation function(PACF)，描述了各个序列的相关性。
通过 PACF 图可以确定 p

通过 ACF 图可以确定 q

## SARIMA

季节性差分自回归滑动平均模型，seasonal autoregressive integrated moving averaging(SARIMA)，在 ARIMA 模型的基础上进行了季节性调节。
其形式为：SARIMA(p,d,q)(P,D,Q)s，其中P,D,Q为季节参数，s为时间序列周期。

## ETS：指数平滑法

ETS，Exponential Smoothing

由于时间序列数据随时间变化但具有一定的随机性，我们通常希望对数据进行平滑处理。为此，我们将使用 ETS 技术，通过指数方法为过去的数据分配较少的权重。同时将时间序列数据分解为趋势（T）、季节（S）和误差（E）分量。
三种常用 ETS 方法如下：
Linear：双指数平滑；

Additive：三指数平滑；

Multiplicative：三指数平滑。

## 总结
在本文中，单变量预测方法在广告支出数据上表现良好。但这些方法难以组合/合并新的信号（如事件、天气）。同时这些方法对丢失数据也非常敏感，通常不能很好地预测很长一段时间。

In [ ]:
def test_stationarity(timeseries):
    rolmean = timeseries.rolling(window=30).mean()
    rolstd = timeseries.rolling(window=30).std()
    plt.figure(figsize=(14,5))
    sns.despine(left=True)
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best');
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    print ('<Results of Dickey-Fuller Test>')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
        print(dfoutput)

In [ ]:
df1=df.resample('D', how=np.mean)

In [ ]:
test_stationarity(df1.Spend.dropna())

In [ ]:
fit1 = sm.tsa.statespace.SARIMAX(train.Spend, order=(7, 1, 2), seasonal_order=(0, 1, 2, 7)).fit(use_boxcox=True)
test['SARIMA'] = fit1.predict(start="2019-07-23", end="2019-09-23", dynamic=True)
plt.figure(figsize=(16, 8)) plt.plot(train['Spend'], label='Train')
plt.plot(test['Spend'], label='Test')
plt.plot(test['SARIMA'], label='SARIMA')
plt.legend(loc='best')
plt.show()

In [ ]:
fit1 = ExponentialSmoothing(np.asarray(train['Spend']) ,seasonal_periods=7 ,trend='add', seasonal='add').fit(use_boxcox=True)
test['Holt_Winter'] = fit1.forecast(len(test))
plt.figure(figsize=(16,8))
plt.plot( train['Spend'], label='Train')
plt.plot(test['Spend'], label='Test')
plt.plot(test['Holt_Winter'], label='Holt_Winter')
plt.legend(loc='best')
plt.show()
